In [1]:
from __future__ import print_function
import pcl
import pcl.pcl_visualization
import numpy as np
import laspy
#from copy import deepcopy
#from pclpy import pcl
#import pcl
#from pcl import pcl_visualization
#import pcl.pcl_visualization
import matplotlib.pyplot as plt

In [2]:
#convert las file to pcd file
def las2pcd():
    input_file="D:/TUM File/semester3/photogrammetry project/data/training/labelledLocalCRS/DEBY_LOD2_4959462/DEBY_LOD2_4959462.las"
    point_cloud=laspy.read(input_file)
    
    points = np.vstack((point_cloud.x, point_cloud.y, point_cloud.z)).transpose()
    colors = np.vstack((point_cloud.red, point_cloud.green, point_cloud.blue)).transpose()
    cloud = pcl.PointCloud.PointXYZRGBA().from_array(np.array(points, dtype=np.float32),colors)
    address="D:/TUM File/semester3/photogrammetry project/data/training/labelledLocalCRS/DEBY_LOD2_4959462/DEBY_LOD2_4959462_pcl.pcd"
    pcl.io.savePCDFileASCII(address,cloud)

In [3]:

#visualize pcd file
def readCloud(): 
    file='D:/TUM File/semester3/photogrammetry project/data/training/labelledLocalCRS/DEBY_LOD2_4959462/DEBY_LOD2_4959462_pcl.pcd'
    #file='D:/TUM File/semester3/photogrammetry project/data/training/labelledGlobalCRS/DEBY_LOD2_4959462_pcl.pcd'
    cloud = pcl.load(file)
    # Centred the data
    centred = cloud - np.mean(cloud, 0)
    # print(centred)
    ptcloud_centred = pcl.PointCloud()
    ptcloud_centred.from_array(centred)
    visual = pcl.pcl_visualization.CloudViewing()

    # PointXYZ
    visual.ShowMonochromeCloud(ptcloud_centred, b'cloud')
    v = True
    while v:
        v = not(visual.WasStopped())
        

In [ ]:
#las2pcd()       
readCloud()

file='D:/TUM File/semester3/photogrammetry project/data/training/labelledLocalCRS/DEBY_LOD2_4959462/DEBY_LOD2_4959462_pcl.pcd'
cloud0 = pcl.load(file)


In [ ]:
#####point segmentation
def regionGrowing():
    ne=cloud0.make_NormalEstimation()
    kdtree0=cloud0.make_kdtree()
    # ne.set_KSearch(50)
    ne.set_RadiusSearch(200.0)
    # set_KSearch()和set_RadiusSearch()不要同时设置，不然计算法线结果为0
    ne.set_SearchMethod(kdtree0)
    normal0=ne.compute()
    
    #区域生长
    start = time.time()
    
    rg=cloud0.make_RegionGrowing(searchRadius=400.0)# 同理，r和k只要填一个就行
    rg.set_MaxClusterSize(100000)
    rg.set_MinClusterSize(0)
    rg.set_NumberOfNeighbours(200)
    rg.set_InputNormals(normal0)
    rg.set_SearchMethod(kdtree0)
    rg.set_SmoothnessThreshold(2 / 180.0 * 3.14)
    rg.set_CurvatureThreshold(0.5)
    rg_result=rg.Extract()
    
    end = time.time()
    print("Execution Time: ", end - start)
    
    # 取最大的num个区域
    rg_list = sorted(rg_result,key = lambda i:len(i),reverse=True)  
    arr=cloud0.to_array()
    
    ##渐变可视化最大的num个区域
    vs=viewer.PCLVisualizering
    vss1=viewer.PCLVisualizering()#初始化一个对象  
    vs.SetBackgroundColor(vss1, 255, 255, 255)
    
    num=10
    for i in range(0, num):
        cloud_i=pcl.PointCloud(arr[rg_list[i]])
        visualcolor0 = viewer.PointCloudColorHandleringCustom(cloud_i, 255, 0, int(255/num*i))
        vs.AddPointCloud_ColorHandler(vss1,cloud_i,visualcolor0,id=b'cloud'+str(i).encode(),viewport=0)
    
    while not vs.WasStopped(vss1):
        vs.Spin(vss1)

    
    return 0


regionGrowing()